# Custom Architectures

The aim of this tutorial script is to show, how you can integrate your own neural network architectures into the MIScnn pipeline.

### MIScnn Pipeline

As explained in more detail in the wiki, we need 4 core components in the pipeline:
- Data I/O  
- Data Augmentation (optional)
- Preprocessor
- Neural Network Model

These classes handle all required steps for medical image segmentation and can be extensively customized. All classes, except the Data Augmentation class, use switchable interfaces which results into high configurability and offers simple integration of user-defined solutions.

For our custom model, only the Neural Network class will be interesting for us. Before we can start, let's initialize a little testing pipeline in which we can deploy our custom architectures.

In [1]:
# Import libraries
from miscnn import Data_IO, Preprocessor, Neural_Network
from miscnn.data_loading.interfaces import Dictionary_interface
import numpy as np

# Create 2D testing data set (grayscale image with binary mask)
dataset2D = dict()
for i in range(0, 100):
    img = np.random.rand(16, 16) * 255
    img = img.astype(int)
    seg = np.random.rand(16, 16) * 2
    seg = seg.astype(int)
    dataset2D["TEST.sample_" + str(i)] = (img, seg)
    
# Initialize Dictionary IO Interface
io_interface2D = Dictionary_interface(dataset2D, classes=2, three_dim=False)

# Create the Data I/O object
data_io = Data_IO(io_interface2D, input_path="")

# Create and configure the Preprocessor class
pp = Preprocessor(data_io, data_aug=None, batch_size=2, analysis="fullimage")

# Get sample list
sample_list = data_io.get_indiceslist()

### Neural Network Class

After we successfully setup our dummy pipeline, we can have a closer look on the Neural Network class.

This class provides functionality for handling all model methods. This class runs the whole pipeline and uses a Preprocessor instance to obtain batches.
With an initialized Neural Network model instance, it is possible to run training, prediction and evaluations.

The core function of the Neural Network class is to define the model architecture and its loss function for training. By default, the Neural Network class uses a standard U-Net architecture and the Tversky loss function. 

Before we begin with our custom architecture, let's have a look on the Neural Network architecture parameter and how to use it.

In [2]:
# Import standard U-Net architecture
from miscnn.neural_network.architecture.unet import UNet_standard

# Initialize the imported standard U-Net architecture
current_architecture = UNet_standard()

# Create a deep learning neural network model with a standard U-Net architecture
nn = Neural_Network(preprocessor=pp, architecture=current_architecture)

In [3]:
# Output the model summary from Keras
nn.model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 9248        batch_normalization[0][0]        
_______________________________________________________________________________________

### Creating an Architecture

Now, we can start implementing our custom architecture.

MIScnn utilizes the subpackage Keras of Tensorflow for designing/defining architectures.
Therefore, we have to use the Keras layers.

You can find an extensive documentation of the Keras layers in the Tensorflow API:  
https://www.tensorflow.org/api_docs/python/tf/keras/layers


In order to create a new architecture, which is compatibile with MIScnn, it is required to create a architecture class. The architecture class has to contain the following three class functions:


```
class Abstract_Architecture():
    __init__                Object initialization function
    create_model_2D:        Creating a 2D Keras model (for 2D data)
    create_model_3D:        Creating a 3D Keras model (for 3D data)
```

In Python code, our custom architecture class will look like this:

In [4]:
# Library Imports
from miscnn.neural_network.architecture.abstract_architecture import Abstract_Architecture

# My Architecture Class
class My_Architecture(Abstract_Architecture):

    def __init__(self):
        pass
    
    def create_model_2D(self, input_shape, n_labels=2):
        return None
    
    def create_model_3D(self, input_shape, n_labels=2):
        return None

Depending on your dataset (settings of your Data IO interface), MIScnn will call the create_model_2D or create_model_3D function to obtain the corresponding model.

The __init__ function will be called by yourself when setting up the pipeline (Exactly as we did it previously with the imported standard U-Net). This gives you the opportunity to pass your own variables into your architecture class. A common use case would be to implement boolean tags if you want to include dropout or batch normalization layers in your architecture.  
BUT: This is all optional. In our example, we don't specify anything init function, because you probably won't need it.

So, let's start with creating a custom architecture. Due to our dataset is 2D, we only need a 2D architecture.  
Therefore, we just fill the create_model_2D function.

In [5]:
# Library Imports
from miscnn.neural_network.architecture.abstract_architecture import Abstract_Architecture
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose

# My Architecture Class
class My_Architecture(Abstract_Architecture):

    def __init__(self):
        pass
    
    def create_model_2D(self, input_shape, n_labels=2):
        # Define Input Layer
        input_layer = Input(input_shape)

        # Define some architecture
        convE = Conv2D(64, 3, activation="relu", padding="same")(input_layer)
        convE = Conv2D(64, 3, activation="relu", padding="same")(convE)
        poolE = MaxPooling2D(pool_size=(2, 2))(convE)

        convM = Conv2D(128, 3, activation="relu", padding="same")(poolE)
        convM = Conv2D(128, 3, activation="relu", padding="same")(convM)

        convT = Conv2DTranspose(64, 3, strides=(2, 2), padding="same")(convM)
        convD = concatenate([convT, convE])
        convD = Conv2D(64, 3, activation="relu", padding="same")(convD)
        convD = Conv2D(64, 3, activation="relu", padding="same")(convD)

        # Define Output Layer
        output_layer = Conv2D(n_labels, (1,1), padding="same", activation="softmax")(convD)

        # Define Keras Model (associated with input and output layers)
        model = Model(inputs=[input_layer], outputs=[output_layer])
        
        # Return the model to the pipeline
        return model

    def create_model_3D(self, input_shape, n_labels=2):
        return None

Be aware to watch out that the last tensor (output_layer) has the same dimension on the last axis as the number of classes (n_labels) in your segmentation dataset.  
For example, if we have a 2D segmentation with 10 classes the output shape must be: (x, y, 10)

Congratulations. That's it!
We have successfully created a custom architecture for MIScnn.

### Using a custom Architecture

Now, let's use our new custom architecture by integrating it in the MIScnn pipeline.  
The proceeding to load the model is identical as before with the imported standard U-Net architecture.

In [6]:
# Initialize our new architecture
new_architecture = My_Architecture()

# Create a deep learning neural network model with our custom architecture
nn = Neural_Network(preprocessor=pp, architecture=new_architecture)

In [7]:
# Output the model summary from Keras
nn.model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, None, None, 6 640         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, None, None, 6 36928       conv2d_19[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, None, None, 6 0           conv2d_20[0][0]                  
_______________________________________________________________________________________

In [8]:
# Start training the model
nn.train(sample_list, epochs=10)

Epoch 1/10
50/50 [==============================] - 0s 8ms/step - loss: 1.0219 - dice_soft: 0.4890
Epoch 2/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9835 - dice_soft: 0.5082
Epoch 3/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9713 - dice_soft: 0.5144
Epoch 4/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9603 - dice_soft: 0.5198
Epoch 5/10
50/50 [==============================] - 0s 8ms/step - loss: 0.9591 - dice_soft: 0.5205
Epoch 6/10
50/50 [==============================] - 0s 8ms/step - loss: 0.9527 - dice_soft: 0.5237
Epoch 7/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9345 - dice_soft: 0.5328
Epoch 8/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9177 - dice_soft: 0.5412
Epoch 9/10
50/50 [==============================] - 0s 7ms/step - loss: 0.9021 - dice_soft: 0.5489
Epoch 10/10
50/50 [==============================] - 0s 7ms/step - loss: 0.8906 - dice_soft: 0.5547


### Final Words

Here, I presented the workflow of integrating custom architectures into the MIScnn pipeline.  
I hope to give you an intention on how MIScnn works and how you can integrate/use MIScnn in your research projects.

For further questions or suggestions, please do not hesitate to get in contact with me.  
Also, if you want to contribute to MIScnn do not hesitate! :)

Thanks for reading,  
Dominik Müller